# Azure AI Search နှင့် NVIDIA NIM နှင့် LlamaIndex ပေါင်းစပ်မှု

ဒီ notebook မှာ NVIDIA ရဲ့ AI မော်ဒယ်တွေကို LlamaIndex နဲ့ပေါင်းစပ်ပြီး အထူးကောင်းမွန်တဲ့ Retrieval-Augmented Generation (RAG) pipeline တစ်ခုကို ဖန်တီးပုံကို ပြသပေးမှာဖြစ်ပါတယ်။ NVIDIA ရဲ့ LLMs နဲ့ embeddings ကို အသုံးပြုပြီး Azure AI Search ကို vector store အနေနဲ့ ပေါင်းစပ်ကာ RAG ကိုလုပ်ဆောင်ပြီး ရှာဖွေမှုရဲ့ အရည်အသွေးနဲ့ ထိရောက်မှုကို မြှင့်တင်ပေးပါမယ်။

## အကျိုးကျေးဇူးများ
- **Scalability**: NVIDIA ရဲ့ အကြီးစားဘာသာစကားမော်ဒယ်တွေကို Azure AI Search နဲ့ပေါင်းစပ်ပြီး အကျယ်ပြန့်နဲ့ ထိရောက်တဲ့ ရှာဖွေမှုကို ရရှိနိုင်ပါမယ်။
- **Cost Efficiency**: ထိရောက်တဲ့ vector storage နဲ့ hybrid search နည်းလမ်းတွေကို အသုံးပြုပြီး ရှာဖွေမှုနဲ့ retrieval ကို စျေးနှုန်းသက်သာစွာ အကောင်းဆုံးလုပ်ဆောင်နိုင်ပါမယ်။
- **High Performance**: အားကောင်းတဲ့ LLMs ကို vectorized search နဲ့ပေါင်းစပ်ပြီး မြန်ဆန်ပြီး တိကျတဲ့ အဖြေတွေကို ရရှိနိုင်ပါမယ်။
- **Quality**: LLM ရဲ့ အဖြေတွေကို သက်ဆိုင်တဲ့ စာရွက်စာတမ်းတွေကို ရှာဖွေပြီး အခြေခံထားကာ ရှာဖွေမှုရဲ့ အရည်အသွေးကို ထိန်းသိမ်းနိုင်ပါမယ်။

## လိုအပ်ချက်များ
- 🐍 Python 3.9 သို့မဟုတ် အထက်
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key (NVIDIA NIM microservices မှတဆင့် NVIDIA ရဲ့ LLMs နဲ့ Embeddings ကို အသုံးပြုရန်)

## ဖော်ပြမည့် အင်္ဂါရပ်များ
- ✅ NVIDIA LLM Integration (ကျွန်တော်တို့ [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) ကို အသုံးပြုပါမယ်)
- ✅ NVIDIA Embeddings (ကျွန်တော်တို့ [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) ကို အသုံးပြုပါမယ်)
- ✅ Azure AI Search ရဲ့ အဆင့်မြင့် Retrieval Modes
- ✅ LlamaIndex ကို အသုံးပြု၍ စာရွက်စာတမ်းများကို Index လုပ်ခြင်း
- ✅ Azure AI Search နဲ့ LlamaIndex ကို NVIDIA LLMs နဲ့ပေါင်းစပ်ပြီး RAG လုပ်ဆောင်ခြင်း

စတင်လိုက်ရအောင်!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## တပ်ဆင်ခြင်းနှင့် လိုအပ်ချက်များ  
Python version >3.10 ကို အသုံးပြု၍ Python ပတ်ဝန်းကျင်တစ်ခု ဖန်တီးပါ။

## စတင်အသုံးပြုခြင်း!  


NVIDIA AI Foundation မော်ဒယ်များကို အသုံးပြုရန် `NVIDIA_API_KEY` တစ်ခုလိုအပ်ပါသည်။
1) [NVIDIA](https://build.nvidia.com/explore/discover) တွင် အခမဲ့အကောင့်တစ်ခု ဖန်တီးပါ။
2) သင်ရွေးချယ်လိုသော မော်ဒယ်ကို နှိပ်ပါ။
3) Input အောက်တွင် Python tab ကို ရွေးချယ်ပြီး **Get API Key** ကို နှိပ်ပါ၊ ထို့နောက် **Generate Key** ကို နှိပ်ပါ။
4) ဖန်တီးထားသော key ကို ကူးယူပြီး NVIDIA_API_KEY အဖြစ် သိမ်းဆည်းပါ။ ထို့နောက် သင်သည် endpoint များကို အသုံးပြုနိုင်ပါပြီ။


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG ကို LLM နှင့် Embedding အသုံးပြု၍ နမူနာ
### 1) LLM ကို စတင်မောင်းနှင်ခြင်း
`llama-index-llms-nvidia` ဟာ NVIDIA ရဲ့ LLM connector အနေနဲ့ အသုံးပြုနိုင်ပြီး NVIDIA API catalog မှာ ရရှိနိုင်တဲ့ compatible models တွေကို ချိတ်ဆက်ပြီး ထုတ်လုပ်နိုင်စေပါတယ်။ Chat completion models တွေစာရင်းကို ဒီမှာကြည့်နိုင်ပါတယ် - https://build.nvidia.com/search?term=Text-to-Text

ဒီနေရာမှာ **mixtral-8x7b-instruct-v0.1** ကို အသုံးပြုပါမယ်။


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Embedding ကို Initialize လုပ်ခြင်း  
`llama-index-embeddings-nvidia` ဟာ NVIDIA ရဲ့ Embeddings connector အနေနဲ့လည်း သိပ်ပြီးကျယ်ကျယ်ပြန့်ပြန့်အသုံးပြုနိုင်တဲ့ API catalog မှာရှိတဲ့ compatible models တွေနဲ့ချိတ်ဆက်ပြီး ထုတ်လုပ်နိုင်စေပါတယ်။ ကျွန်တော်တို့ `nvidia/nv-embedqa-e5-v5` ကို embedding model အဖြစ်ရွေးချယ်ခဲ့ပါတယ်။ Text embedding models များစာရင်းကို ဒီမှာကြည့်နိုင်ပါတယ် - https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) သင့်ဒေတာအပေါ်မေးခွန်းများမေးရန် Query Engine တစ်ခုဖန်တီးပါ

ဤသည်မှာ Azure AI Search တွင် သန့်စင်သောဗက်တာရှာဖွေရေးကို အသုံးပြုပြီး၊ ကျွန်ုပ်တို့၏ LLM (Phi-3.5-MOE) သို့ အခြေခံထားသော တုံ့ပြန်မှုကို ပေါင်းစပ်သည့် query တစ်ခုဖြစ်သည်။


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Azure AI Search တွင် hybrid search ကို အသုံးပြုသော query တစ်ခုမှာ ဒီမှာရှိပါတယ်။


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ဗက်တာ ရှာဖွေမှု ချုပ်ခြင်း  
LLM ၏ တုံ့ပြန်မှုသည် ရုရှား၏ စတော့ရှယ်ယာဈေးကွက်နှင့် ပတ်သက်သော အဓိက စီးပွားရေးဆိုင်ရာ ရလဒ်များကို အရင်းအမြစ်စာသားတွင် ဖော်ပြထားသည့်အတိုင်း တိကျစွာ ဖော်ထုတ်ထားသည်။ အထူးသဖြင့် ရုရှား စတော့ရှယ်ယာဈေးကွက်သည် တန်ဖိုး၏ ၄၀% ကို ဆုံးရှုံးခဲ့ပြီး လက်ရှိအခြေအနေကြောင့် ရောင်းဝယ်မှုများကို ရပ်ဆိုင်းခဲ့သည်ဟု ဖော်ပြထားသည်။ ဤတုံ့ပြန်မှုသည် အရင်းအမြစ်တွင် ဖော်ပြထားသည့် အချက်များနှင့် လိုက်လျောညီထွေရှိပြီး ရုရှား၏ လုပ်ဆောင်မှုများနှင့် ပိတ်ဆို့မှုများကြောင့် စတော့ရှယ်ယာဈေးကွက်အပေါ် ဖြစ်ပေါ်လာသည့် သက်ရောက်မှုများကို LLM မှ တိကျစွာ ဖော်ထုတ်ပြီး အကျဉ်းချုပ်ထားသည်ကို ပြသသည်။

#### အရင်းအမြစ် Node များအပေါ် မှတ်ချက်  
အရင်းအမြစ် node များတွင် နိုင်ငံတကာ ပိတ်ဆို့မှုများကြောင့် ရုရှားကြုံတွေ့ခဲ့ရသည့် စီးပွားရေးဆိုင်ရာ ရလဒ်များကို အသေးစိတ် ဖော်ပြထားသည်။ စာသားတွင် ရုရှား စတော့ရှယ်ယာဈေးကွက်သည် တန်ဖိုး၏ ၄၀% ကို ဆုံးရှုံးခဲ့ပြီး ရောင်းဝယ်မှုများကို ရပ်ဆိုင်းခဲ့သည်ဟု ဖော်ပြထားသည်။ ထို့အပြင် ရူဘယ်ငွေကြေးတန်ဖိုးကျဆင်းမှုနှင့် ရုရှား စီးပွားရေးစနစ်၏ ကျယ်ပြန့်သော သီးခြားမှုများကဲ့သို့သော အခြားသော စီးပွားရေးဆိုင်ရာ ရလဒ်များကိုလည်း ဖော်ပြထားသည်။ LLM ၏ တုံ့ပြန်မှုသည် ဤ node များမှ အရေးကြီးသော အချက်များကို ထိရောက်စွာ ချုပ်ဆောင်နိုင်ခဲ့ပြီး စတော့ရှယ်ယာဈေးကွက်အပေါ် သက်ရောက်မှုကို အဓိကထား၍ မေးခွန်းအရ တုံ့ပြန်ခဲ့သည်။


ယခု Hybrid Search က အခြေခံထားသော အဖြေမပေးနိုင်တဲ့ မေးခွန်းတစ်ခုကို ကြည့်ကြရအောင်:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ဟိုက်ဘရစ်ရှာ့ချ်: LLM အဖြေခွဲခြမ်းစိတ်ဖြာခြင်း
ဟိုက်ဘရစ်ရှာ့ချ် ဥပမာအတွင်း LLM အဖြေက ရုရှားက ယူကရိန်းကို ကျူးကျော်ခဲ့တဲ့ တိကျတဲ့ရက်စွဲကို ပေးထားတဲ့အကြောင်းအရာမှာ မဖော်ပြထားကြောင်း ပြသနေပါတယ်။ ဒီအဖြေက LLM က ရင်းမြစ်စာရွက်စာတမ်းများထဲက အချက်အလက်များကို အသုံးချနေပြီး၊ စာသားအတွင်းမှာ တိကျတဲ့အသေးစိတ်မပါဝင်ကြောင်းကိုလည်း အသိအမှတ်ပြုနေကြောင်း ဖော်ပြပါတယ်။

ဒီအဖြေက ရုရှားရဲ့ တိုက်ခိုက်မှုနှင့်ဆိုင်သော ဖြစ်ရပ်များကို အကြောင်းအရာအတွင်းမှာ ဖော်ပြထားကြောင်း မှန်ကန်စွာ ဖော်ထုတ်ပေးနိုင်သော်လည်း၊ ကျူးကျော်မှုရက်စွဲကို တိကျစွာ ဖော်ပြမထားကြောင်းကိုလည်း သေချာစွာ ဖော်ပြထားပါတယ်။ ဒါက LLM ရဲ့ ပေးထားတဲ့အချက်အလက်ကို နားလည်နိုင်စွမ်းနှင့် အကြောင်းအရာအတွင်းရှိ အချက်အလက်မပြည့်စုံမှုကို အသိအမှတ်ပြုနိုင်စွမ်းကို ပြသနေပါတယ်။ အချက်အလက်မပြည့်စုံတဲ့အခါမှာ ပြင်ပရင်းမြစ်များ သို့မဟုတ် သမိုင်းမှတ်တမ်းများကို ရှာဖွေဖို့ အသုံးပြုသူကို အထောက်အကူပြုနိုင်အောင် သတိပေးနိုင်တဲ့ အရည်အသွေးကိုလည်း ဖော်ပြထားပါတယ်။

### ရင်းမြစ် Node များခွဲခြမ်းစိတ်ဖြာခြင်း
ဟိုက်ဘရစ်ရှာ့ချ် ဥပမာအတွင်း ရင်းမြစ် node များမှာ ရုရှားရဲ့ လုပ်ရပ်များအပေါ် အမေရိကန်ရဲ့ တုံ့ပြန်မှုကို ဆွေးနွေးထားတဲ့ မိန့်ခွန်းတစ်ခုမှ အပိုင်းအစများ ပါဝင်ပါတယ်။ ဒီ node များက ကျယ်ပြန့်တဲ့ နိုင်ငံတကာနိုင်ငံရေးအကျိုးသက်ရောက်မှုနှင့် ရုရှားရဲ့ ကျူးကျော်မှုအပေါ် အမေရိကန်နှင့် မိတ်ဖက်နိုင်ငံများက လုပ်ဆောင်ခဲ့တဲ့ အဆင့်ဆင့်များကို အဓိကထားဖော်ပြထားပါတယ်။ သို့သော် တိကျတဲ့ ကျူးကျော်မှုရက်စွဲကို မဖော်ပြထားပါဘူး။ ဒါက LLM အဖြေနှင့် ကိုက်ညီနေပြီး၊ အကြောင်းအရာအတွင်းမှာ တိကျတဲ့ရက်စွဲအချက်အလက်မပါဝင်ကြောင်း မှန်ကန်စွာ ဖော်ထုတ်ပေးနိုင်မှုကို ပြသနေပါတယ်။


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: LLM Response Analysis
Hybrid w/Reranking နမူနာတွင် LLM ရဲ့ အဖြေက အခန်းကဏ္ဍကို ပိုမိုရှင်းလင်းစေဖို့ အကြောင်းအရာအပိုင်းကို ထည့်သွင်းပေးထားပါတယ်။ ဥပမာအားဖြင့် အဆိုပြုခဲ့တဲ့နေ့မတိုင်မီ ရက်ခြောက်ရက်အကြာမှာ ဖြစ်ပွားခဲ့တဲ့ အဖြစ်အပျက်ကို ဖော်ပြထားတာပါ။ ဒါက LLM က အဆိုပြုခဲ့တဲ့နေ့ကို အတိအကျ သိရန်လိုအပ်သော်လည်း အဆိုပြုခဲ့တဲ့အချိန်ကို အခြေခံပြီး ကျူးကျော်မှုရက်စွဲကို ခန့်မှန်းနိုင်စွမ်းရှိကြောင်း ပြသပါတယ်။

ဒီအဖြေက အကြောင်းအရာအချက်အလက်များကို အသုံးပြုပြီး ပိုမိုအသေးစိတ်ဖြစ်စေတဲ့ အဖြေတစ်ခုကို ပေးနိုင်စွမ်းရှိကြောင်း ပြသပါတယ်။ ဒါဟာ reranking ရဲ့ အားသာချက်ကို ဖော်ပြပြီး LLM က ပိုမိုသက်ဆိုင်တဲ့ အချက်အလက်များကို ရယူပြီး အဓိကအချက်အလက် (ဥပမာ - ကျူးကျော်မှုရက်စွဲ) ကို ပိုမိုနီးကပ်စွာ ခန့်မှန်းနိုင်စွမ်းရှိကြောင်း ပြသပါတယ်။

### Source Nodes Analysis
ဒီနမူနာမှာ Source Nodes တွေက ရုရှားရဲ့ ကျူးကျော်မှုအချိန်ကို ရည်ညွှန်းထားပြီး အဆိုပြုခဲ့တဲ့နေ့မတိုင်မီ ရက်ခြောက်ရက်အကြာမှာ ဖြစ်ပွားခဲ့ကြောင်း ဖော်ပြထားပါတယ်။ ရက်စွဲကို အတိအကျ မဖော်ပြထားသော်လည်း Nodes တွေက အချိန်ဆိုင်ရာ အကြောင်းအရာကို ပေးထားပြီး LLM က ပိုမိုအသေးစိတ်ဖြစ်စေတဲ့ အဖြေတစ်ခုကို ပေးနိုင်စွမ်းရှိစေပါတယ်။ ဒီအချက်အလက်ကို ထည့်သွင်းထားတာက reranking က အကြောင်းအရာကို ရယူပြီး ခန့်မှန်းနိုင်စွမ်းကို တိုးတက်စေပြီး ပိုမိုတိကျတဲ့ အဖြေတစ်ခုကို ပေးနိုင်စွမ်းရှိကြောင်း ပြသပါတယ်။


**မှတ်ချက်:**
ဒီ notebook မှာ NVIDIA API Catalog ထဲက NVIDIA NIM microservices တွေကို အသုံးပြုထားပါတယ်။
အထက်မှာဖော်ပြထားတဲ့ API တွေ `NVIDIA (llms)`, `NVIDIAEmbedding`, နဲ့ [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview) ပါဝင်ပါတယ်။ အထက်ဖော်ပြထားတဲ့ API တွေဟာ self-hosted microservices တွေကိုလည်း ပံ့ပိုးပေးနိုင်ပါတယ်။

**ဥပမာ:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတည်သော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူပညာရှင်များမှ ဘာသာပြန်ဆိုမှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ဆိုမှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပာယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
